In [1]:
pip install Faker


     |████████████████████████████████| 1.0MB 10.1MB/s 


In [2]:
pip install nmt_utils

ERROR: Could not find a version that satisfies the requirement nmt_utils (from versions: none)
ERROR: No matching distribution found for nmt_utils


In [3]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline


Using TensorFlow backend.


In [0]:
fake = Faker()

# We need to seed these guys. For some reason I always use 101
Faker.seed(101)
random.seed(101)

In [0]:
FORMATS = ['short', 
           'medium', 
           'medium',
           'medium',
           'long', 
           'long',
           'long',
           'long',
           'long',
           'full', 
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'dd/MM/YYY',
           'EE d, MMM YYY',
           'EEEE d, MMMM YYY']

In [6]:
for format in FORMATS:
    print('%s => %s' %(format, format_date(fake.date_object(), format=format, locale='en')))

short => 7/19/09
medium => Apr 3, 1983
medium => Sep 11, 2006
medium => May 29, 1994
long => October 15, 2001
long => April 20, 1973
long => February 24, 2015
long => April 7, 2004
long => August 6, 1984
full => Monday, December 20, 2010
full => Friday, February 1, 1985
full => Sunday, August 20, 1989
d MMM YYY => 14 Jan 2003
d MMMM YYY => 13 February 2017
d MMMM YYY => 14 June 1984
d MMMM YYY => 23 May 1992
d MMMM YYY => 22 October 1999
d MMMM YYY => 15 October 1974
dd/MM/YYY => 16/05/1987
EE d, MMM YYY => Sat 26, Feb 1983
EEEE d, MMMM YYY => Wednesday 17, December 1980


In [0]:
def random_date():
    dt = fake.date_object()

    try:
        date = format_date(dt, format=random.choice(FORMATS), locale='en')
        human_readable = date.lower().replace(',', '')
        machine_readable = dt.isoformat()

    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

In [0]:
def create_dataset(m):
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    
    for i in tqdm(range(m)):
        h, m, _ = random_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    # We also add two special chars, <unk> for unknown characters, and <pad> to add padding at the end
    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v: k for k, v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

In [9]:
m = 30000
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(m)

100%|██████████| 30000/30000 [00:01<00:00, 28450.31it/s]


In [10]:
dataset[:10]


[('18/01/1976', '1976-01-18'),
 ('april 18 2000', '2000-04-18'),
 ('26 july 2006', '2006-07-26'),
 ('saturday december 10 1994', '1994-12-10'),
 ('15 january 1983', '1983-01-15'),
 ('feb 5 2002', '2002-02-05'),
 ('9 march 1992', '1992-03-09'),
 ('april 2 1986', '1986-04-02'),
 ('sat 18 may 2013', '2013-05-18'),
 ('june 26 1997', '1997-06-26')]

In [11]:
human_vocab

{' ': 0,
 '/': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11,
 '<pad>': 35,
 '<unk>': 34,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'r': 27,
 's': 28,
 't': 29,
 'u': 30,
 'v': 31,
 'w': 32,
 'y': 33}

In [12]:
machine_vocab


{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [13]:
inv_machine_vocab


{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

In [0]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

In [0]:
def string_to_int(string, length, vocab):
    string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep

In [16]:
string_to_int('September 10, 1978', 30, human_vocab)


[28,
 16,
 26,
 29,
 16,
 23,
 13,
 16,
 27,
 0,
 3,
 2,
 0,
 3,
 11,
 9,
 10,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35]

In [17]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (30000, 30)
Y.shape: (30000, 10)
Xoh.shape: (30000, 30, 36)
Yoh.shape: (30000, 10, 11)


In [18]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 18/01/1976
Target date: 1976-01-18

Source after preprocessing (indices): [ 3 10  1  2  3  1  3 11  9  8 35 35 35 35 35 35 35 35 35 35 35 35 35 35
 35 35 35 35 35 35]
Target after preprocessing (indices): [ 2 10  8  7  0  1  2  0  2  9]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [0]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

In [0]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [0]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation='softmax')

In [0]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model([X, s0, c0], outputs)
    return model

In [0]:
mod = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))


In [24]:
mod.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 36)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17664       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]               

In [0]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [27]:
mod.fit([Xoh, s0, c0], outputs, epochs=30, batch_size=100)


Epoch 1/30
30000/30000 [==============================] - 50s 2ms/step - loss: 9.9830 - dense_3_loss: 1.9072 - dense_3_accuracy: 0.9203 - dense_3_accuracy_1: 0.9258 - dense_3_accuracy_2: 0.7255 - dense_3_accuracy_3: 0.3977 - dense_3_accuracy_4: 0.8907 - dense_3_accuracy_5: 0.6412 - dense_3_accuracy_6: 0.3537 - dense_3_accuracy_7: 0.8582 - dense_3_accuracy_8: 0.4779 - dense_3_accuracy_9: 0.2841
Epoch 2/30
30000/30000 [==============================] - 42s 1ms/step - loss: 3.2913 - dense_3_loss: 0.8909 - dense_3_accuracy: 0.9888 - dense_3_accuracy_1: 0.9899 - dense_3_accuracy_2: 0.8844 - dense_3_accuracy_3: 0.8237 - dense_3_accuracy_4: 0.9999 - dense_3_accuracy_5: 0.9761 - dense_3_accuracy_6: 0.8341 - dense_3_accuracy_7: 0.9998 - dense_3_accuracy_8: 0.7678 - dense_3_accuracy_9: 0.6836
Epoch 3/30
30000/30000 [==============================] - 42s 1ms/step - loss: 2.0380 - dense_3_loss: 0.5228 - dense_3_accuracy: 0.9904 - dense_3_accuracy_1: 0.9910 - dense_3_accuracy_2: 0.8939 - dense_3_ac

In [28]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = mod.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 1979-05-03
source: 5 April 09
output: 2009-04-05
source: 21th of August 2016
output: 2016-08-22
source: Tue 10 Jul 2007
output: 2007-07-10
source: Saturday May 9 2018
output: 2018-05-09
source: March 3 2001
output: 2001-03-03
source: March 3rd 2001
output: 2001-03-03
source: 1 March 2001
output: 2001-03-01


In [0]:
mod.save('dates_model.h5')
